# 미니 프로젝트 3차 회의 준비자료

이번에 해야할 것
    1. 의료 데이터 적합도 체크 (charge가 없을 때)
    2. 모델별 평가지표 비교, 정리 (6개의 분류모델 사용)
    3. 내가 맡은 모델 , SVM과 KNN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [2]:
data=pd.read_csv("C:/Users/SAMSUNG/Desktop/framingham3.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4185 entries, 0 to 4184
Data columns (total 16 columns):
male               4185 non-null int64
currentSmoker      4185 non-null int64
cigsPerDay         4185 non-null float64
BPMeds             4185 non-null int64
prevalentStroke    4185 non-null int64
prevalentHyp       4185 non-null int64
diabetes           4185 non-null int64
totChol            4185 non-null float64
sysBP              4185 non-null float64
diaBP              4185 non-null float64
BMI                4185 non-null float64
heartRate          4185 non-null float64
glucose            4185 non-null float64
TenYearCHD         4185 non-null int64
BMI_rank           4185 non-null float64
age                4185 non-null object
dtypes: float64(8), int64(7), object(1)
memory usage: 523.2+ KB


In [3]:
data=data.drop(['BMI'],axis='columns')         #bmi변수는 범주화해서 사용할 것이므로 bmi_rank만을 사용하기 위해 제거
#get_dummies 함수
data = pd.get_dummies(data,columns=['age'],prefix='나이')
#hundred_students = pd.get_dummies(hundred_students, columns=['지역'], prefix='지역')
data = pd.get_dummies(data,columns=['BMI_rank'],prefix='BMI')
print(data.head(10))
data.info()

   male  currentSmoker  cigsPerDay  BPMeds  prevalentStroke  prevalentHyp  \
0     0              1        15.0       0                0             0   
1     1              0         0.0       0                0             0   
2     0              0         0.0       0                0             0   
3     1              1        20.0       0                0             0   
4     0              1        30.0       0                0             1   
5     0              1        23.0       0                0             0   
6     0              0         0.0       0                0             1   
7     0              0         0.0       0                0             0   
8     0              1        20.0       0                0             0   
9     1              0         0.0       0                0             1   

   diabetes  totChol  sysBP  diaBP   ...      glucose  TenYearCHD  나이_30대  \
0         0    242.0  111.0   70.0   ...     0.404817           0       1  

In [4]:
# 독립변수와 종속변수로 Split:
X = data.drop(['TenYearCHD'], axis=1)
y = data['TenYearCHD']

SVM과 KNN 모두 feature scaling이 필요

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
X_sig = scaler.fit_transform(X)

# Data split:

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sig, y, test_size=0.30, random_state=1)

In [7]:
#SVM, KNN

# KNN Classifier:

In [8]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [9]:
knn.fit(X_train, y_train)

y_pred_train = knn.predict(X_train)
y_prob_train = knn.predict_proba(X_train)[:,1]

y_pred = knn.predict(X_test)
y_prob = knn.predict_proba(X_test)[:,1]

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      1051
           1       0.39      0.12      0.18       205

    accuracy                           0.83      1256
   macro avg       0.62      0.54      0.54      1256
weighted avg       0.77      0.83      0.79      1256



In [10]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint                    #random search
knn = KNeighborsClassifier()

params = {'n_neighbors': sp_randint(1,25),
        'p': sp_randint(1,5)}

rand_search_knn = RandomizedSearchCV(knn, param_distributions=params,
                                 cv=3, random_state=1)
rand_search_knn.fit(X_train, y_train)
print(rand_search_knn.best_params_)

{'n_neighbors': 19, 'p': 1}


In [11]:
# KNN using best parameters

knn = KNeighborsClassifier(**rand_search_knn.best_params_)

knn.fit(X_train, y_train)

y_pred_train = knn.predict(X_train)
y_prob_train = knn.predict_proba(X_train)[:,1]

y_pred = knn.predict(X_test)
y_prob = knn.predict_proba(X_test)[:,1]

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      1051
           1       0.50      0.01      0.03       205

    accuracy                           0.84      1256
   macro avg       0.67      0.51      0.47      1256
weighted avg       0.78      0.84      0.77      1256



In [12]:
from sklearn.metrics import recall_score,precision_score,classification_report,roc_auc_score,roc_curve
# calculate AUC
knn_auc = roc_auc_score(y_test, y_prob) ; knn_auc

0.656065071592676

# SVM Classifier:

In [13]:
from sklearn.svm import SVC

In [14]:
#grid search for optimum parameters
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
svm_clf = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, cv=10)

In [15]:
# train the model
svm_clf.fit(X_train,y_train)
svm_clf.best_params_ 

{'C': 0.001, 'gamma': 0.001}

In [16]:
# predictions
svm_predict = svm_clf.predict(X_test)

In [22]:
print(classification_report(y_test,svm_predict))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      1051
           1       0.00      0.00      0.00       205

    accuracy                           0.84      1256
   macro avg       0.42      0.50      0.46      1256
weighted avg       0.70      0.84      0.76      1256



C:\Users\SAMSUNG\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
# ROC curve and AUC 
probs = svm_clf.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
svm_auc = roc_auc_score(y_test, probs) ;svm_auc

0.6363695435241697